In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def print_ai_message_details(ai_message):
    print("📩 AIMessage Details:")
    print(f"Content:\n  {ai_message.content}\n")

    print("Additional Kwargs:")
    for k, v in ai_message.additional_kwargs.items():
        print(f"  {k}: {v}")
    
    # print("\nResponse Metadata:")
    # for k, v in ai_message.response_metadata.items():
    #     print(f"  {k}: {v}")
    
    print(f"\nID: {ai_message.id}")

    # print("\nUsage Metadata:")
    # for k, v in ai_message.usage_metadata.items():
    #     print(f"  {k}: {v}")


In [3]:
prompt = "How will the weather be in Munich today?"
# prompt = "What is the capital of France?"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
llm_message = llm.invoke(prompt)
print_ai_message_details(llm_message)

📩 AIMessage Details:
Content:
  I'm unable to provide real-time weather updates, including the current weather in Munich. I recommend checking a reliable weather website or app for the latest information.

Additional Kwargs:
  refusal: None

ID: run--2489e7b0-8c87-4334-965f-fea62392d0ce-0


In [6]:
def print_human_message_details(messages):
    for i, msg in enumerate(messages):
        print(f"\n--- Message {i+1} ({msg.__class__.__name__}) ---")
        
        if hasattr(msg, 'content'):
            print(f"Content: {msg.content}")
        
        if hasattr(msg, 'additional_kwargs') and msg.additional_kwargs:
            print("Additional Kwargs:")
            for k, v in msg.additional_kwargs.items():
                print(f"  {k}: {v}")
        
        # if hasattr(msg, 'response_metadata') and msg.response_metadata:
        #     print("Response Metadata:")
        #     for k, v in msg.response_metadata.items():
        #         print(f"  {k}: {v}")

        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print("Tool Calls:")
            for call in msg.tool_calls:
                print(f"  - Tool Name: {call.get('name')}")
                print(f"    Args: {call.get('args')}")
                print(f"    ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        if hasattr(msg, 'id'):
            print(f"ID: {msg.id}")
        
        # if hasattr(msg, 'usage_metadata') and msg.usage_metadata:
        #     print("Usage Metadata:")
        #     for k, v in msg.usage_metadata.items():
        #         print(f"  {k}: {v}")


In [7]:
# print_ai_message_details(llm_message)

## Same python code

```python
class WeatherInput(BaseModel):
    city: str

# Define the actual function
def fake_weather_api(city: str) -> str:
    return f"The weather in {city} is always sunny in this fake API."

# Register as a StructuredTool
weather_tool = StructuredTool.from_function(
    func=fake_weather_api,
    name="fake_weather_api",
    description="Check the weather in a specified city.",
    args_schema=WeatherInput
)

tools = [weather_tool]
```

In [8]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return f"{city}: Sunny, 22°C"


tools = [fake_weather_api]

In [9]:
tools

[StructuredTool(name='fake_weather_api', description='Check the weather in a specified city.\n\nArgs:\n    city (str): The name of the city where you want to check the weather.\n\nReturns:\n    str: A description of the current weather in the specified city.', args_schema=<class 'langchain_core.utils.pydantic.fake_weather_api'>, func=<function fake_weather_api at 0x0000017E54D0A980>)]

In [10]:
llm_with_tools = llm.bind_tools(tools)

In [11]:
result = llm_with_tools.invoke(prompt)
print_ai_message_details(result)

📩 AIMessage Details:
Content:
  

Additional Kwargs:
  tool_calls: [{'id': 'call_R33OhbmW6mM9PDNjPsatla1m', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]
  refusal: None

ID: run--367c1492-02f1-4699-a230-e9d8abdbe0e6-0


## Step 1 User Prompt => HumanMessage

In [12]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(prompt)
]

print_human_message_details(messages)


--- Message 1 (HumanMessage) ---
Content: How will the weather be in Munich today?
ID: None


## Step 2 Binding Tools w User Prompt => LLM


In [13]:
llm_output = llm_with_tools.invoke(messages)
print_ai_message_details(llm_output)



📩 AIMessage Details:
Content:
  

Additional Kwargs:
  tool_calls: [{'id': 'call_4URFHZWFKCFj1568H1XyGcpX', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]
  refusal: None

ID: run--6c5e1fe9-519d-4f7c-8fb1-36eedc55eed2-0


In [14]:
messages.append(llm_output)
print_human_message_details(messages)


--- Message 1 (HumanMessage) ---
Content: How will the weather be in Munich today?
ID: None

--- Message 2 (AIMessage) ---
Content: 
Additional Kwargs:
  tool_calls: [{'id': 'call_4URFHZWFKCFj1568H1XyGcpX', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]
  refusal: None
Tool Calls:
  - Tool Name: fake_weather_api
    Args: {'city': 'Munich'}
    ID: call_4URFHZWFKCFj1568H1XyGcpX
    Type: tool_call
ID: run--6c5e1fe9-519d-4f7c-8fb1-36eedc55eed2-0


## Step 3 Call Tools and Bind Results to HumanMessage

In [15]:
tool_mapping = {"fake_weather_api": fake_weather_api}

for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

print_human_message_details(messages)


--- Message 1 (HumanMessage) ---
Content: How will the weather be in Munich today?
ID: None

--- Message 2 (AIMessage) ---
Content: 
Additional Kwargs:
  tool_calls: [{'id': 'call_4URFHZWFKCFj1568H1XyGcpX', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]
  refusal: None
Tool Calls:
  - Tool Name: fake_weather_api
    Args: {'city': 'Munich'}
    ID: call_4URFHZWFKCFj1568H1XyGcpX
    Type: tool_call
ID: run--6c5e1fe9-519d-4f7c-8fb1-36eedc55eed2-0

--- Message 3 (ToolMessage) ---
Content: Munich: Sunny, 22°C
ID: None


## Step 4 Binding Messages => LLM => AIMessage

In [16]:
final_result = llm_with_tools.invoke(messages)
print_ai_message_details(final_result)

📩 AIMessage Details:
Content:
  The weather in Munich today is sunny with a temperature of 22°C.

Additional Kwargs:
  refusal: None

ID: run--5ba13a57-a215-49c1-8fa2-bda584871bb1-0
